In [57]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Build an open lakehouse on Google Cloud

<table align="left">
  <td><a href="https://colab.research.google.com/github/GoogleCloudPlatform/devrel-demos/blob/main/data-analytics/lakehouse-webinar/open_lakehouse.ipynb"><img src="https://avatars.githubusercontent.com/u/33467679?s=200&v=4" width="32px" alt="Colab logo"> Run in Colab</a></td>
  <td><a href="https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/data-analytics/lakehouse-webinar/open_lakehouse.ipynb"><img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px" alt="GitHub logo"> View on GitHub</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/devrel-demos/blob/main/data-analytics/lakehouse-webinar/open_lakehouse.ipynb"><img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> Open in Vertex AI Workbench</a></td>
  <td><a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/data-analytics/lakehouse-webinar/open_lakehouse.ipynb"><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35"> Open in BQ Studio</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fdevrel-demos%2Fblob%2Fmain%data-analytics%2Ficeberg_webinar%2Fopen_lakehouse.ipynb"><img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"> Open in Colab Enterprise</a></td>
</table>

TheLook, a fictional ecommerce company, is looking to migrate and modernize their open lakehouse on Google Cloud. They are also looking to learn more about customer behavior, specifically around returned orders.

In this notebook, you set up an lakehouse using [Apache Spark](spark.apache.org), [Apache Iceberg](https://iceberg.apache.org/), [BigLake](https://cloud.google.com/biglake), [BigQuery](https://cloud.google.com/bigquery), [Dataplex](https://cloud.google.com/dataplex), [Cloud Storage](https://cloud.google.com/storage) and [Vertex AI](https://cloud.google.com/vertex-ai).


## Setup

Configure environment variables. Provide your project ID and a [region](https://cloud.google.com/bigquery/docs/locations#regions) to store your resources, such as `us-central1`. **Note**: this tutorial will not work with [multi-regions](https://cloud.google.com/bigquery/docs/locations#multi-regions).

In [ ]:
PROJECT_ID = "" #@param
LOCATION = "" #@param
WAREHOUSE_BUCKET_NAME = f"{PROJECT_ID}-warehouse"

Create a Cloud Storage bucket to house staging data.

In [ ]:
from google.cloud import storage

STAGING_BUCKET_NAME=f"{PROJECT_ID}-staging"

storage_client = storage.Client()
if not storage_client.bucket(STAGING_BUCKET_NAME).exists():
    storage_client.create_bucket(STAGING_BUCKET_NAME, location=LOCATION)

Create a Cloud Storage bucket for your warehouse.

In [ ]:
WAREHOUSE_BUCKET_NAME = f"{PROJECT_ID}-warehouse"

if not storage_client.bucket(WAREHOUSE_BUCKET_NAME).exists():
    storage_client.create_bucket(WAREHOUSE_BUCKET_NAME, location=LOCATION)

Copy a [Delta Lake](https://delta.io/) table to your new bucket, which contains a list of products.

In [ ]:
source_prefix = "lakehouse_notebook/delta"
source_bucket = storage_client.bucket("data-analytics-demos")
destination_bucket = storage_client.bucket(STAGING_BUCKET_NAME)

blobs = source_bucket.list_blobs(prefix=source_prefix)

for blob in blobs:
    new_blob_name = "delta" + blob.name[len(source_prefix):]
    source_bucket.copy_blob(blob, destination_bucket, new_blob_name)


Create the dataset that will store your data.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE SCHEMA IF NOT EXISTS `lakehouse`
OPTIONS (
  location = 'YOUR_LOCATION', -- Replace with your location
  description = 'Store lakehouse data'
)
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Create an empty BigQuery table that will store product data.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE TABLE IF NOT EXISTS lakehouse.products
(
  id INT64,
  cost FLOAT64,
  category STRING,
  name STRING,
  brand STRING,
  retail_price FLOAT64,
  department STRING,
  sku STRING,
  distribution_center_id INT64
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Populate the table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
LOAD DATA INTO lakehouse.products
FROM FILES (
  uris=['gs://data-analytics-demos/thelook_ecommerce/products/*'],
  format='PARQUET'
)
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Create a BigQuery connection to connect to Google Cloud resources such as Vertex AI and Google Cloud Storage.

In [ ]:
import subprocess

CONNECTION_ID = "resource_connection"
command = [
    'bq',
    'mk',
    '--connection',
    '--location',
    LOCATION,
    '--connection_type=CLOUD_RESOURCE',
    CONNECTION_ID
]

result = subprocess.run(command, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Update the BigQuery Connection with the necessary permissions.

**Note**: The following cell will occasionally fail as the connection service account isn't always available right away after connection creation. The workaround is as follows:

1. Copy the service account email that's printed from the cell below.
2. Go to the [IAM Admin](https://console.cloud.google.com/iam-admin/iam) page and click **Grant Access**
3. In **Add principals** paste the service account email
3. In **Assign roles** add the roles `aipaltform.admin` and `storage.admin`.

In [ ]:
from google.cloud import bigquery_connection_v1

# Get service account used with BigQuery connection
connection_client = bigquery_connection_v1.ConnectionServiceClient()

full_connection_name = connection_client.connection_path(
    PROJECT_ID, LOCATION, CONNECTION_ID
)

connection = connection_client.get_connection(name=full_connection_name)
sa_email = connection.cloud_resource.service_account_id
print(f"BigQuery Connection service account: {sa_email}")

roles = ["aiplatform.admin", "storage.admin"]

for role in roles:
  command = [
      'gcloud',
      'projects',
      'add-iam-policy-binding',
      PROJECT_ID,
      '--member',
      f'serviceAccount:{sa_email}',
      '--role',
      f'roles/{role}'
  ]

  result = subprocess.run(command, capture_output=True, text=True)
  print(result.stdout)
  print(result.stderr)

## Create a BigLake external table for Delta Lake.

Create an external table to manage your Delta Lake data.

In [ ]:
%%bigquery

CREATE EXTERNAL TABLE IF NOT EXISTS `lakehouse.users_delta`
WITH CONNECTION `YOUR_LOCATION.resource_connection` -- Update location
OPTIONS (
  format = 'DELTA_LAKE',
  uris = ['gs://YOUR_PROJECT_ID-staging/delta/users'] -- Update project ID
);

Query this table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
select * from lakehouse.users_delta
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

## Create BigLake Tables for Apache Iceberg

Create a [BigLake Table for Apache Iceberg](https://cloud.google.com/bigquery/docs/iceberg-tables).

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE TABLE lakehouse.order_history_iceberg_imt_mstx (
  id INT64,
  order_id INT64,
  user_id INT64,
  product_id INT64,
  inventory_item_id INT64,
  status STRING,
  created_at TIMESTAMP,
  shipped_at TIMESTAMP,
  delivered_at TIMESTAMP,
  returned_at TIMESTAMP,
  sale_price FLOAT64
)
WITH CONNECTION `YOUR_LOCATION.resource_connection` -- Update location
OPTIONS (
  file_format = 'PARQUET',
  table_format = 'ICEBERG',
  storage_uri = 'gs://YOUR_PROJECT_ID-warehouse/order_history_iceberg_imt_mstx' -- Update project ID
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Populate the table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
LOAD DATA INTO lakehouse.order_history_iceberg_imt_mstx
FROM FILES (
  uris=['gs://data-analytics-demos/thelook_ecommerce/order_items/*'],
  format='PARQUET'
)

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Query the table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  lakehouse.order_history_iceberg_imt_mstx;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

With BigLake, you can query across table formats. Join the Iceberg order history table with the Delta Lake table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT * FROM lakehouse.order_history_iceberg_imt_mstx i
LEFT JOIN lakehouse.users_delta d
ON i.user_id = d.id;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

Create a second table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE TABLE lakehouse.order_history_returns_iceberg_imt_mstx (
  id INT64,
  order_id INT64,
  user_id INT64,
  product_id INT64,
  inventory_item_id INT64,
  status STRING,
  created_at TIMESTAMP,
  shipped_at TIMESTAMP,
  delivered_at TIMESTAMP,
  returned_at TIMESTAMP,
  sale_price FLOAT64
)
WITH CONNECTION `YOUR_LOCATION.resource_connection` -- Update location
OPTIONS (
  file_format = 'PARQUET',
  table_format = 'ICEBERG',
  storage_uri = 'gs://YOUR_PROJECT_ID-warehouse/order_history_returns_iceberg_imt_mstx' -- Update project ID
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

BigLake Tables for Apache Iceberg support [multi-statement transactions](https://cloud.google.com/bigquery/docs/transactions). With this, you simultaneously insert rows from one table into another, and then delete the rows from the source table.

**Note:** You'll need to use an [allowlisted](https://cloud.google.com/bigquery/docs/iceberg-tables#use_multi-statement_transactions) project to use this feature. To proceed without this feature, delete `BEGIN TRANSACTION;` and `COMMIT TRANSACTION;` below, which runs the jobs synchronously.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
BEGIN TRANSACTION;

-- Insert rows into order_history_returns_iceberg_imt_mstx
INSERT INTO lakehouse.order_history_returns_iceberg_imt_mstx
SELECT * FROM lakehouse.order_history_iceberg_imt_mstx
WHERE status = "Returned";

-- Delete the matching records from order_history_returns_iceberg_imt_mstx
DELETE lakehouse.order_history_iceberg_imt_mstx
WHERE status = "Returned";

COMMIT TRANSACTION;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

You can also discover and search for these tables in [Dataplex](console.cloud.google.com/dataplex.dp-search-nl).

## Connect to an interactive serverless Spark session

Create a connection to [Google Cloud Serverless for Apache Spark](https://cloud.google.com/products/serverless-spark), which provides you a serverless Spark runtime to execute jobs against. This connection utilizes [Spark Connect](https://spark.apache.org/docs/latest/spark-connect-overview.html). Learn more about [supported configuration parameters](https://cloud.google.com/bigquery/docs/use-spark).

Configure the runtime to use [Lightning Engine for Apache Spark](https://cloud.google.com/products/lightning-engine), which can potentially accelerate Spark jobs by up to 4.3x.

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session

session = Session()

catalog_name = "demo_catalog"

# Configure the Spark runtime to use Lighting Engine
session.runtime_config.properties = {
    'spark.dataproc.runtimeEngine': 'native',
    'spark.dataproc.driver.compute.tier': 'premium',
    'spark.dataproc.executor.compute.tier': 'premium'
}

Configure the runtime to access a BigLake metastore with the [Iceberg REST Catalog](https://cloud.google.com/bigquery/docs/blms-rest-catalog). The Iceberg Rest Catalog supports accessing Iceberg tables in BigLake metastore from any compatible runtime engine via REST calls.

In [ ]:
# Configure the runtime to use the Iceberg REST Catalog
session.runtime_config.properties.update({
    'spark.sql.extensions': 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions',
    'spark.sql.defaultCatalog': f'{catalog_name}',
    f'spark.sql.catalog.{catalog_name}': 'org.apache.iceberg.spark.SparkCatalog',
    f'spark.sql.catalog.{catalog_name}.type': 'rest',
    f'spark.sql.catalog.{catalog_name}.uri': 'https://biglake.googleapis.com/iceberg/v1/restcatalog',
    f'spark.sql.catalog.{catalog_name}.warehouse': f'gs://{WAREHOUSE_BUCKET_NAME}',
    f'spark.sql.catalog.{catalog_name}.header.x-goog-user-project': f'{PROJECT_ID}',
    f'spark.sql.catalog.{catalog_name}.rest.auth.type': 'org.apache.iceberg.gcp.auth.GoogleAuthManager',
    f'spark.sql.catalog.{catalog_name}.io-impl': 'org.apache.iceberg.gcp.gcs.GCSFileIO',
    f'spark.sql.catalog.{catalog_name}.rest-metrics-reporting-enabled': 'false',
})

Create the serverless Spark session. After executing the cell, click **VIEW SESSION DETAILS** to view the Spark UI panel.

In [ ]:
# Create the serverless Spark runtime.
spark = DataprocSparkSession.builder.dataprocSessionConfig(session).getOrCreate()

Use Gemini to generate code to load an iceberg table into Spark.

**Prompt**: load the table lakehouse.order_history_returns_iceberg_imt_mstx using Spark and show me some of the data

In [ ]:
# prompt: load the table lakehouse.order_history_returns_iceberg_imt_mstx using spark and show me some of the data

# NOTE: Pyspark code generation is currently in PREVIEW.
df = spark.read.format("bigquery").option("table", "lakehouse.order_history_returns_iceberg_imt_mstx").load()
df.show()

Register the returns table as a temporary table also.

**Prompt:** register lakehouse.order_history_returns_iceberg_imt_mstx as a temp table named returns

In [ ]:
# prompt: register lakehouse.order_history_returns_iceberg_imt_mstx as a temp table named returns

# NOTE: Pyspark code generation is currently in PREVIEW.
df.createOrReplaceTempView("returns")

Load the BigQuery table `products` and register it as a SparkSQL view.

In [ ]:
spark.read.format("bigquery").load(f"lakehouse.products").createOrReplaceTempView("products")

Create a namespace for your Iceberg tables.

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS spark;")

spark.sql("USE spark;")

Create a new Iceberg table.

In [ ]:
spark.sql("CREATE OR REPLACE TABLE returns_by_category (id int, name string, count string) USING ICEBERG;")

Show the table in the namespace.

In [ ]:
spark.sql("SHOW TABLES").show()

Insert into this table.

In [ ]:
spark.sql("""INSERT INTO returns_by_category
  SELECT
    c.id,
    name,
    COUNT(*)
  FROM products c
  LEFT JOIN returns AS returns
  ON c.id = returns.id
  group by c.id, c.name;
;""")

Query the table using Spark.

In [ ]:
spark.sql("SELECT * FROM returns_by_category").show()

You can query this new table from BigQuery also.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT * FROM `YOUR_PROJECT_ID-warehouse>spark.returns_by_category`; -- Update project ID
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

## Use Gemini to extend data richness

Using the `AI.GENERATE` function in BigQuery, you can pass data from your BigQuery tables to Gemini.

Use `AI.GENERATE` to generate brief product descriptions for the returned products.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  name,
  AI.GENERATE(
    ("Give a short description of ", name),
    connection_id => 'YOUR_LOCATION.resource_connection', -- Update location
    endpoint => 'gemini-2.5-flash').result
FROM
  `YOUR_PROJECT_ID-warehouse>spark.returns_by_category` -- Update project_id

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

# BigQuery ML

Create a logistic regression model to predict return behavior based on order and user data.

You union the two Iceberg orders tables back together and join them with your Delta Lake users table.

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL `lakehouse.returns_model`
OPTIONS(model_type='LOGISTIC_REG',
        input_label_cols=['returned']) AS
SELECT
  product_id,
  age,
  postal_code,
  traffic_source,
  IF(status = "Returned", 0, 1) AS returned
FROM (
  SELECT * FROM `lakehouse.order_history_iceberg_imt_mstx`
  UNION ALL
  SELECT * FROM `lakehouse.order_history_returns_iceberg_imt_mstx`
) orders

LEFT JOIN
  `lakehouse.users_delta` users
ON
  orders.user_id = users.id

Evaluate the model.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL `lakehouse.returns_model`)
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

# Cleaning Up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial by uncommenting below:

In [ ]:
# Stop Spark session
# spark.stop()

# Delete all BigQuery tables + dataset
#!bq rm -r -f -d lakehouse

# Delete storage buckets
#!gcloud storage rm --recursive gs://YOUR_PROJECT_ID-staging
#!gcloud storage rm --recursive gs://YOUR_PROJECT_ID-warehouse
